In [9]:
%autosave 0
import numpy 
import matplotlib
import pickle
import pandas as pd
import numpy as np 
import tensorflow as tf
from pandas import read_csv
from matplotlib import pyplot
from pandas.plotting import scatter_matrix 
from numpy import set_printoptions 
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.compose import ColumnTransformer

set_printoptions(precision=0)
pd.set_option("display.precision", 2)
pd.option_context('display.float_format', '{:0f}'.format);

seed = 15
tf.random.set_seed(seed)
numpy.random.seed(seed)

Autosave disabled


In [45]:
# Importation du fichier csv dans un dataframe
# df_train_full = read_csv( './dataset-template.csv', index_col=0)
df_train_full = read_csv( './csv/realstate-data.csv', index_col=0)

In [46]:
print(df_train_full.shape)
df_train_full.head(5)

(7200, 11)


,city,county,district,area_m2,ground_m2,nb_room,nb_bedroom,pool,cellar,garage,output
ref,,,,,,,,,,,
/fr/properties/30731a-133656.htm,Saint-Maurin,Lot-et-Garonne,Aquitaine,100.0,NaN,5.0,3.0,NaN,NaN,NaN,210000
/fr/properties/88486a-95148cj265.htm,Castelnau-de-Médoc,Gironde,Aquitaine,328.0,3700.0,9.0,4.0,NaN,NaN,True,840000
/fr/properties/44945a-m728.htm,Peyrehorade,Landes,Aquitaine,814.0,100000.0,19.0,8.0,True,True,NaN,1950000
/fr/properties/32488a-2534.htm,Bourlens,Lot-et-Garonne,Aquitaine,406.0,13200.0,14.0,10.0,NaN,NaN,NaN,975200
/fr/properties/32488a-2557.htm,Monsempron-Libos,Lot-et-Garonne,Aquitaine,99.0,35.0,4.0,3.0,NaN,NaN,NaN,101520


In [47]:
df_train_full.describe()

,area_m2,ground_m2,nb_room,nb_bedroom,output
count,6880.00,5.74e+03,6311.00,6221.00,7.20e+03
mean,313.85,2.45e+04,7.10,4.52,4.53e+05
std,1052.33,3.63e+05,3.77,2.34,6.30e+05
min,1.00,1.00e+00,2.00,2.00,8.00e+03
25%,125.00,1.11e+03,5.00,3.00,1.98e+05
50%,180.00,3.00e+03,6.00,4.00,3.11e+05
75%,280.00,9.48e+03,8.00,5.00,4.98e+05
max,29760.00,2.69e+07,70.00,50.00,1.60e+07


In [48]:
Y_trainfull = df_train_full['output']
X_trainfull = df_train_full.drop('output', axis=1)

In [49]:
print(Y_trainfull.shape)
print(X_trainfull.shape)

(7200,)
(7200, 10)


# Traitement des variables catégorielles

In [28]:
cat_columns = ['city', 'county', 'district']

In [29]:
categorical_pipe = Pipeline([
    ('onehot', OneHotEncoder())
])

# Traitement des valeurs manquantes

In [17]:
df_train_full.isna().any()

city          False
county        False
district      False
area_m2        True
ground_m2      True
nb_room        True
nb_bedroom     True
pool           True
cellar         True
garage         True
output        False
dtype: bool

In [31]:
boolean_columns = ['pool', 'cellar', 'garage']
area_column = ['area_m2']
ground_column = ['ground_m2']
nb_room_column = ['nb_room']
nb_bedroom_column = ['nb_bedroom']

In [32]:
boolean_pipe = Pipeline([
    ('simpleimputer0', SimpleImputer(missing_values='NaN', strategy='constant', fill_value=0)),
    ('simpleimputer1', SimpleImputer(missing_values='True', strategy='constant', fill_value=1))
])

In [33]:
area_pipe = Pipeline([
    ('simpleimputer', SimpleImputer(missing_values='NaN', strategy='median'))
])

In [34]:
ground_pipe = Pipeline([
    ('simpleimputer', SimpleImputer(missing_values='NaN', strategy='constant', fill_value=0))
])

In [38]:
room_pipe = Pipeline([
    ('simpleimputer', SimpleImputer(missing_values='NaN', strategy='constant', fill_value=1))
])

In [39]:
bedroom_pipe = Pipeline([
    ('simpleimputer', SimpleImputer(missing_values='NaN', strategy='constant', fill_value=1))
])

# Construction du pipeline de pré-traitement

In [40]:
preprocess_pipe = ColumnTransformer([
    ('cat', categorical_pipe, cat_columns),
    ('bool', boolean_pipe, boolean_columns),
    ('area', area_pipe, area_na_columns),
    ('ground', ground_pipe, ground_na_columns),
    ('room', room_pipe, nb_room_columns),
    ('bedroom', bedroom_pipe, nb_bedroom_columns),
])

# Choix du modèle et intégration dans la pipeline

In [41]:
model = LinearRegression()

In [42]:
full_pipe = Pipeline(
    [('pp', preprocess_pipe),
     ('reg', model)]
)

# Séparations des données en ensemble d'entraînement et de validation

In [50]:
X_train, X_val, Y_train, Y_val = train_test_split(X_trainfull, Y_trainfull, random_state = seed)

In [51]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(5400, 10)
(5400,)
(1800, 10)
(1800,)


# Création modèle

In [52]:
 full_pipe.fit(X_train, Y_train);

ValueError: Input contains NaN

In [29]:
y_train_pred = full_pipe.predict(X_train)
y_val_pred = full_pipe.predict(X_val)

In [34]:
mae_train = round(metrics.mean_absolute_error(Y_train, y_train_pred), 0)
mae_val = round(metrics.mean_absolute_error(Y_val, y_val_pred), 0)
print('MAE sur train : ', mae_train, '€')
print('MAE sur val : ', mae_val, '€')

MAE sur train :  24833.0 €
MAE sur val :  38000.0 €


In [37]:
pickle.dump(model, open( "model.file", "wb" ))